In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D,Activation, SpatialDropout2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D
from keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers
import tensorflow as tf

In [8]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 10
l = 12
num_filter = 36 #added 24 more filters
compression = 0.5 
dropout_rate = 0.2

In [3]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [7]:
densenet_model = Sequential()

pretrained_model = tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_shape=(32,32,3),
    pooling='max',
    classes=10,
    classifier_activation="softmax",
)

for layer in pretrained_model.layers:
    layer.trainable=False

densenet_model.add(pretrained_model)
densenet_model.add(Flatten())
densenet_model.add(Dense(3072, activation='relu'))
densenet_model.add(Dense(2048, activation='relu'))
densenet_model.add(Dense(10, activation='softmax'))

74836368/74836368 [==============================] - 22s 0us/step


In [20]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)

In [21]:
densenet_model.compile(optimizer=SGD(0.01, momentum = 0.7), loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
from keras.callbacks import ModelCheckpoint, CSVLogger
ckpt = ModelCheckpoint('saved_model/densenet_model.hdf5')
csv = CSVLogger('saved_model/densenet_log.csv')

In [23]:
history = densenet_model.fit(datagen.flow(x_train, y_train, batch_size), validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size, callbacks = [csv, ckpt])
densenet_model.save_weights('saved_model/10epochs_densenet_cifar10.h5')

Epoch 1/10
391/391 [==============================] - 277s 684ms/step - loss: 1.9674 - accuracy: 0.3957 - val_loss: 1.5527 - val_accuracy: 0.4589
Epoch 2/10
391/391 [==============================] - 267s 683ms/step - loss: 1.5174 - accuracy: 0.4579 - val_loss: 1.4293 - val_accuracy: 0.4967
Epoch 3/10
391/391 [==============================] - 278s 711ms/step - loss: 1.4568 - accuracy: 0.4798 - val_loss: 1.4227 - val_accuracy: 0.4954
Epoch 4/10
391/391 [==============================] - 257s 658ms/step - loss: 1.4186 - accuracy: 0.4963 - val_loss: 1.3426 - val_accuracy: 0.5214
Epoch 5/10
391/391 [==============================] - 251s 642ms/step - loss: 1.3941 - accuracy: 0.5039 - val_loss: 1.4010 - val_accuracy: 0.5008
Epoch 6/10
391/391 [==============================] - 246s 629ms/step - loss: 1.3754 - accuracy: 0.5115 - val_loss: 1.4417 - val_accuracy: 0.4992
Epoch 7/10
391/391 [==============================] - 261s 667ms/step - loss: 1.3479 - accuracy: 0.5190 - val_loss: 1.3103 -

In [24]:
train_mse = densenet_model.evaluate(x_train, y_train, verbose=0)
test_mse = densenet_model.evaluate(x_test, y_test, verbose=0)

In [25]:
print('Train loss : %.3f %% , Test loss : %.3f %%' % (train_mse[0]*100, test_mse[0]*100))
print('Train accuracy : %.3f %% , Test accuracy : %.3f %%' % (train_mse[1]*100, test_mse[1]*100))

Train loss : 124.990 % , Test loss : 132.159 %
Train accuracy : 55.220 % , Test accuracy : 52.990 %
